# Распознавание собак и кошек на изображениях с помощью признаков, извлеченных предварительно обученной нейронной сетью VGG16

**Источник данных** - соревнования Kaggle [Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/data).

Для распознавания используется предварительно обученная сверточная нейронная сеть VGG16.

Перед использованием необходимо скачать и подготовить данные для обучения, проверки и тестирования. Можно использовать пример в ноутбуке data_preparation

In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam
import numpy as np

C:\Users\sozyk\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 10
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750

## Загружаем предварительно обученную нейронную сеть

In [3]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

## Готовим генераторы данных для извлечения признаков

In [4]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [5]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

Found 17500 images belonging to 2 classes.


In [6]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

Found 3750 images belonging to 2 classes.


In [7]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

Found 3750 images belonging to 2 classes.


## Генерируем признаки и сохраняем их в файлы

Признаки для изображений из набора данных для обучения

In [8]:
features_train = vgg16_net.predict_generator(
        train_generator, nb_train_samples // batch_size)

In [9]:
np.save(open('features_train.npy', 'wb'), features_train)

Признаки для изображений из проверочного набора данных

In [10]:
features_val = vgg16_net.predict_generator(
        val_generator, nb_validation_samples // batch_size)

In [11]:
np.save(open('features_val.npy', 'wb'), features_val)

Признаки для изображений из тестового набора данных

In [12]:
features_test = vgg16_net.predict_generator(
        test_generator, nb_test_samples // batch_size)

In [17]:
np.save(open('features_test.npy', 'wb'), features_test)

Генерируем метки для трех наборов данных

In [18]:
labels_train =  np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

In [19]:
labels_val =  np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

In [20]:
labels_test =  np.array(
        [0] * (nb_test_samples // 2) + [1] * (nb_test_samples // 2))

## Загружаем признаки из файлов

In [3]:
features_train = np.load(open('features_train.npy', 'rb'))
features_val = np.load(open('features_val.npy', 'rb'))
features_test = np.load(open('features_test.npy', 'rb'))

## Создаем полносвязную сверточную нейронную сеть для классификации извлеченных из изображения признаков

In [21]:
model = Sequential()
model.add(Flatten(input_shape=features_train.shape[1:]))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [22]:
model.compile(optimizer='Adam',
              loss='binary_crossentropy', metrics=['accuracy'])

Обучаем модель на признаках, которые извлекли из изображений

In [23]:
model.fit(features_train, labels_train,
              epochs=15,
              batch_size=64,
              validation_data=(features_val, labels_val), verbose=2)

Train on 17500 samples, validate on 3750 samples
Epoch 1/15
 - 5s - loss: 0.3228 - acc: 0.8636 - val_loss: 0.2185 - val_acc: 0.9109
Epoch 2/15
 - 4s - loss: 0.2234 - acc: 0.9053 - val_loss: 0.2346 - val_acc: 0.9019
Epoch 3/15
 - 4s - loss: 0.1967 - acc: 0.9171 - val_loss: 0.2090 - val_acc: 0.9157
Epoch 4/15
 - 4s - loss: 0.1881 - acc: 0.9207 - val_loss: 0.2628 - val_acc: 0.8880
Epoch 5/15
 - 4s - loss: 0.1697 - acc: 0.9304 - val_loss: 0.2085 - val_acc: 0.9152
Epoch 6/15
 - 4s - loss: 0.1643 - acc: 0.9322 - val_loss: 0.2128 - val_acc: 0.9147
Epoch 7/15
 - 4s - loss: 0.1476 - acc: 0.9402 - val_loss: 0.2113 - val_acc: 0.9189
Epoch 8/15
 - 4s - loss: 0.1361 - acc: 0.9432 - val_loss: 0.2114 - val_acc: 0.9165
Epoch 9/15
 - 4s - loss: 0.1253 - acc: 0.9488 - val_loss: 0.2218 - val_acc: 0.9187
Epoch 10/15
 - 4s - loss: 0.1125 - acc: 0.9538 - val_loss: 0.2280 - val_acc: 0.9205
Epoch 11/15
 - 4s - loss: 0.1080 - acc: 0.9575 - val_loss: 0.2328 - val_acc: 0.9147
Epoch 12/15
 - 4s - loss: 0.0941 - a

In [24]:
scores = model.evaluate(features_test, labels_test, verbose=1)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

3750/3750 [==============================] - 0s 78us/step
Аккуратность на тестовых данных: 91.25%
